# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션 ID 설정하고, 대화를 진행합니다. (멀티 턴 대화)  
LLM 모델이 과거 대화를 알지(기억) 못 하는 상황을 만드세요

In [ ]:
# 1. 모듈(파일, 라이브러리리) 읽어오기
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from dotenv import load_dotenv

# 2. 환경변수에 설정된 값 읽어오기
load_dotenv()


In [ ]:
# # 3. 파일 읽기
# file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

# with open(file_name, 'r', encoding='utf-8') as file:
#     law = file.read()

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

store = {}

## 4. prompt 생성
template = [
    ('system', '''넌 20년 차 인사과 부장이야 이력서에 빠삭한 사람으로 대답해 줘'''),
    ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 5-1 trim 설정
trimmer = trim_messages(
    max_tokens=60,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human'
)


# 6. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)

## 6-1 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

## 9. 사용자 질문 받기
## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)
## 11. 실행
while True:
    query = input('이력서의 관련된 이야기를 쓰시오')
    if query.upper() == 'S':
        break 
    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '123'}}
    )
    print('\n' + '=' * 50 + '\n') 

In [10]:
store

{'123': InMemoryChatMessageHistory(messages=[])}

In [12]:
get_history(123)

InMemoryChatMessageHistory(messages=[HumanMessage(content='', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 어떤 이력서 관련 질문이나 도움이 필요하신가요? 경험을 바탕으로 최선을 다해 도와드리겠습니다. 궁금한 점이 있다면 말씀해 주세요!', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44'}, id='run-4ac887b4-2338-4ac4-9c4b-70ecfc714c1e-0')])